# Medición de impacto de campañas publicitarias de Uber mediante series de tiempo bayesianas

**Elias Buitrago Bolivar** <br>
*Bogotá, Colombia* <br>
*26/mayo/2021* <br>
*Las actualizaciones de este código se encuentran en mi cuenta [github](https://github.com/ebuitrago/BI/blob/main/ARL_accidentes_trabajo/modelo_AT_arboles.ipynb)*

## Cargar librerías

In [8]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn import metrics, svm
from sklearn.metrics import recall_score
import graphviz
import seaborn as sns

## Cargar datos
Se leen los archivos de datos formato csv. Inicialmente se tienen disponibles seis archivos con los datos de viajes de Uber desde abril hasta septiembre.

In [9]:
col_names = ['No','Fecha.Hora','Lat','Lon','Categoria','Zona']
datos_abril = pd.read_csv("Datos/datos_abril.csv", skiprows=1, names=col_names)
datos_abril = datos_abril.drop(['No'], axis=1, index=None)
datos_mayo = pd.read_csv("Datos/datos_mayo.csv", skiprows=1, names=col_names)
datos_mayo = datos_mayo.drop(['No'], axis=1)
datos_junio = pd.read_csv("Datos/datos_junio.csv", skiprows=1, names=col_names)
datos_junio = datos_junio.drop(['No'], axis=1)
datos_julio = pd.read_csv("Datos/datos_julio.csv", skiprows=1, names=col_names)
datos_julio = datos_julio.drop(['No'], axis=1)
datos_agosto = pd.read_csv("Datos/datos_agosto.csv", skiprows=1, names=col_names)
datos_agosto = datos_agosto.drop(['No'], axis=1)
datos_septiembre = pd.read_csv("Datos/datos_septiembre.csv", skiprows=1, names=col_names)
datos_septiembre = datos_septiembre.drop(['No'], axis=1)

In [ ]:
# datos = datos_abril
# datos = datos.append(datos_mayo)
# datos = datos.append(datos_junio)
# datos = datos.append(datos_julio)
# datos = datos.append(datos_agosto)
# datos = datos.append(datos_septiembre)

## Transformación datos
A partir de los datos iniciales, se hacen transformaciones parar obtener los datos específicos que permitan analizar los resultados de las dos campañas adelantadas por la empresa.

### Datos campaña Jersey City
Se filtran los datos para seleccionar los viajes de la zona Jersey City. Luego, se hace otro filtrado para extraer los viajes en las horas valle: desde la media noche hasta las once. Finalmente se crea una lista con el conteo del número de viajes diarias en el horario de interés para la campaña.

In [ ]:
# jc_data = datos_abril[['Zona','Fecha.Hora']].groupby('Zona', ).get_group('Jersey City').reset_index(drop=True)

In [10]:
def cmp1(datos_mes):
    f=[]
    d=[]
    h=[]
    jc_data = datos_mes[(datos_mes.Zona == 'Jersey City')].reset_index(drop=True)
    for i in range(0,len(jc_data)):
        aux = dt.datetime.strptime(jc_data.iloc[i,0], '%m/%d/%Y %H:%M:%S')
#         aux1 = aux.day
        aux2 = aux.hour
        f.insert(i,aux.strftime('%m/%d'))
#         d.insert(i,aux1)
        h.insert(i,aux2)

    jc_data_cmp1=pd.DataFrame()
    jc_data_cmp1['Fecha'] = f
#     jc_data_cmp1['Dia'] = d
    jc_data_cmp1['Hora'] = h

    jc_data_cmp1 = jc_data_cmp1[(jc_data_cmp1.Hora <= 11)].reset_index(drop=True)
    serie_cmp1=jc_data_cmp1.groupby('Fecha').count()
    print(datos_cmp1_abril.head())
    return serie_cmp1

In [11]:
def cmp1_ref(datos_mes):
    d=[]
    h=[]
    jc_data = datos_mes[(datos_mes.Zona != 'Jersey City')].reset_index(drop=True)
    for i in range(0,len(jc_data)):
        aux = dt.datetime.strptime(jc_data.iloc[i,0], '%m/%d/%Y %H:%M:%S')
        aux1 = aux.day
        aux2 = aux.hour
        d.insert(i,aux1)
        h.insert(i,aux2)

    jc_data_cmp1=pd.DataFrame()
    jc_data_cmp1['Dia'] = d
    jc_data_cmp1['Hora'] = h
    jc_data_cmp1.head()

    jc_data_cmp1 = jc_data_cmp1[(jc_data_cmp1.Hora <= 11)].reset_index(drop=True)
    serie_cmp1_ref=jc_data_cmp1.groupby('Dia').count()    
    return serie_cmp1_ref

In [12]:
#Filtrar datos de viajes en horas valle en la zona Jersey City, para analizar la campaña #1
datos_cmp1_abril = cmp1(datos_abril)
datos_cmp1_mayo = cmp1(datos_mayo)
datos_cmp1_junio = cmp1(datos_junio)
datos_cmp1_julio = cmp1(datos_julio)
datos_cmp1_agosto = cmp1(datos_agosto)
datos_cmp1_septiembre = cmp1(datos_septiembre)

In [ ]:
#Filtrar datos de viajes en horas valle en zonas distintas a Jersey City, como referencias para complementar en análisis de la campaña #1
# from time import time
# start_time = time()
# datos_cmp1Ref_abril = cmp1_ref(datos_abril)
# elapsed_time = time() - start_time
# print("Elapsed time: %0.10f seconds." % elapsed_time)

In [13]:
datos_cmp1Ref_mayo = cmp1_ref(datos_mayo)
datos_cmp1Ref_junio = cmp1_ref(datos_junio)
datos_cmp1Ref_julio = cmp1_ref(datos_julio)
datos_cmp1Ref_agosto = cmp1_ref(datos_agosto)
datos_cmp1Ref1_septiembre = cmp1_ref(datos_septiembre)

In [90]:
datos_cmp1_abril = cmp1(datos_abril)

       Hora
Fecha      
04/01    46
04/02    45
04/03    50
04/04    96
04/05    67


### Datos campaña UberPool
Se filtran los datos para seleccionar los viajes de la categoria UberPool en la zona de Brooklyn. Finalmente se crea una lista con el conteo del número de viajes diarios.

## Análisis campaña #1